<a href="https://colab.research.google.com/github/cdbm/F1DataAnalysis/blob/master/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação 
o seguinte notebook foi feito para fazer predições usando modelos de classificação em cima do dataset da fórmula 1. Escolhemos o dataset de resultados, que contém os resultados de cada piloto em todas as corridas, para fazer nossa classificação. Para a classificação escolhemos a columa positions, que define em que posição o piloto terminou a corrida.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
results=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/results.csv")
races=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/races.csv")

Limpando o dataset para ter apenas pilotos que de fato terminaram as corridas.

In [3]:
results = results[results.position != '\\N']
results = results[results.milliseconds != '\\N']

O código a seguir foi feito para tornar mais possível a classificação. Como depois da limpeza restam poucas linhas (aproximadamente 6 mil) e o dataset não tem tantas features que podem ser usadas para o modelo e existem muitas classes possíveis para se predizer (19), resolvemos então transformar as 19 classes que seriam cada posição de chegada em classes mais abrangentes. O nosso modelo agora busca classificar entre as quatro classes de posições que um piloto pode chegar, entre os 5 primeiros, entre o 6º e o 10º, entre o 11º e o 15º e entre o 16º e 19º.

In [5]:
results.loc[results['position'] <= 5, 'beans'] = 'TOP5'
results.loc[(results['position'] > 5) & (results['position'] <= 10) , 'beans'] = 'SIX_TO_TEN'
results.loc[(results['position'] > 10) & (results['position'] <= 15) , 'beans'] = 'EL_TO_FIFTEEN'
results.loc[(results['position'] > 15) & (results['position'] <= 20) , 'beans'] = 'SIXTEEN_TO_TWENTY'

Adicionando a informação de qual circuito foi o dado resultado


In [6]:
results["circuitId"] = results.raceId.map(races.set_index('raceId')['circuitId'])

In [ ]:
results.head()

Dropando features que ou não podem ser usadas, ou não são úteis ou são úteis até demais para o modelo.

In [8]:
results = results.drop('number', axis=1)
results = results.drop('statusId', axis=1)
results = results.drop('time', axis=1)
results = results.drop('fastestLapTime', axis=1)
results = results.drop('positionOrder', axis=1)
results = results.drop('positionText', axis=1)
results = results.drop('resultId', axis=1)
results = results.drop('fastestLapSpeed', axis=1)
results = results.drop('rank', axis=1)
results = results.drop('fastestLap',axis=1)

In [ ]:
results

Definição de tipos

In [22]:
results.milliseconds = results.milliseconds.astype('int64')
results.beans = results.beans.astype('category')
results.position = results.position.astype("int64")

In [ ]:
results.dtypes

Criação dos Modelos


In [31]:
from sklearn.model_selection import train_test_split
x_results = results.drop('beans', axis=1)
y_results = results['beans']

In [ ]:
x_results

In [ ]:
y_results

In [26]:
Xtrain, Xtest, ytrain, ytest = train_test_split(x_results, y_results, random_state=1)

In [ ]:
ytest

In [27]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=4)             
model.fit(Xtrain, ytrain)                  
prediction = model.predict(Xtest)  
predictions2 = model.predict(Xtrain) 

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest, prediction)
print(prediction.shape)
print(ytest.shape)
print(accuracy)

(1613,)
(1613,)
0.5765654060756354


In [29]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='sgd',random_state=1)
clf.fit(Xtrain, ytrain)
y_pred=clf.predict(Xtest)
y_predTrain=clf.predict(Xtrain) #Fazendo a predição no conjunto de treinamento

In [30]:
print(accuracy_score(ytest, y_pred))
print(accuracy_score(ytrain, y_predTrain))

0.6782393056416615
0.6646001239925604
